<a href="https://colab.research.google.com/github/AnishRSIGM/CombineForecasts/blob/master/combine_forecasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python implementation of

Shah, Anish, Easy Way to Merge Return Forecasts across Securities and Horizons (September 24, 2019). Available at SSRN: https://ssrn.com/abstract=3459184 or http://dx.doi.org/10.2139/ssrn.3459184

In [0]:
import numpy as np

### 1. Create dummy data

In [0]:
def dummy_forecasts(m, n):
  # function creates dummy forecast data to test forecast combination
  # m = # of forecasts
  # n = # of securities
  # returns:
  # start = (m x 1) start periods for forecasts
  # end = (m x 1) end periods for forecasts
  # P = (m x n) linear combinations forecasted
  # y = (m x 1) return forecasts
  # H = (m x m) forecast noise covariance
  # mu = (n x 1) prior mean of 1 period returns
  # C = (n x n) prior covariance of 1 period returns

  start = np.random.randint(low=0,high=10,size=m)
  end = start + np.random.randint(low=1,high=5,size=m)

  s = 0.1
  y = s * np.random.random(size=[m,1])
  I = np.identity(n)
  k = int(np.floor(1.5*m))
  r = np.random.randint(low=0, high=n, size=k)
  P = I[r[:m],:]  # forecast random individual securities
  P[-(k-m):,:] = P[-(k-m):,:] - I[r[m:],:]  # make some spreads
  y[-(k-m):] -= s * 0.5 
  v = np.random.random(n)
  v = np.around(v / v.sum(), 2)
  v[np.argmax(v)] -= v.sum() - 1
  P[0,:] = v # make first entry a portfolio

  Q = np.around(np.random.random(size=[m,m]), 2)
  H = 10. * Q.dot(Q.T)

  Q = np.around(np.random.random(size=[n,n]), 2)
  C = Q.dot(Q.T)
  mu = np.around(0.01*(np.random.random(size=[n,1]) - 0.5), 3)
  return start, end, P, y, H, mu, C

In [0]:
# m = # of forecasts
# n = # of securities
#
# start = (m x 1) start periods for forecasts
# end = (m x 1) end periods for forecasts
# P = (m x n) linear combinations forecasted
# y = (m x 1) return forecasts
# H = (m x m) forecast noise covariance
# mu = (n x 1) prior mean of 1 period returns
# C = (n x n) prior covariance of 1 period returns

m = 5  # number of forecasts
n = 20 # number of securities
start, end, P, y, H, mu, C = dummy_forecasts(m, n)

### 2. Segment objects being forecasted into time segments. Then calculate posterior mean and covariance given the forecasts

In [0]:
# model is y = P x + eps  where eps ~ N(0, H)
# x occurs over the intervals start -> end  and prior x over 1 period ~ N(mu, C)
# want mean and cov of x over different horizons | y

In [0]:
def calculate_posterior_of_segments(start, end, P, H, mu, C, more_horizons=[0.]):

  # more_horizons = (list) of points to consider in addition to start and end
  #               = [0., 5., 21.]  # e.g. to be able to get 1 week and 1 month return forecasts
  m, n = P.shape # num of forecasts x num of securities

  # points in ascending order where time needs to be segmented
  # Tpts = np.unique((start,end))
  Tpts = np.unique([t for x in [start, end, more_horizons] for t in x])

  # put forecast start and end in terms of time markers
  startpt = np.searchsorted(Tpts, start)
  endpt = np.searchsorted(Tpts, end)
  assert np.alltrue(Tpts[startpt] == start)
  assert np.alltrue(Tpts[endpt] == end)

  # break quantities being forecasted into time segments
  # e.g. r(0->T) = r(0->T1) + r(T1->T2) + ... + r(Tk-1->T)
  nseg = len(Tpts) - 1
  nsegvars = n*nseg
  Z = np.zeros((m, nsegvars)) # matrix that will hold forecasts in terms of segments
  nu = np.zeros((nsegvars,1)) # vector that will hold mean for each segment variable
  Omega = np.zeros((nsegvars,nsegvars)) # matrix that will hold cov of segment variables
  for i in range(nseg):
    l = Tpts[i+1] - Tpts[i]   # number of time periods in segment
    sidx = i*n       # start index of variables in time segment
    eidx = sidx + n  # end index of variables in time segment
    nu[sidx:eidx,:] = l * mu            # mean over segment
    Omega[sidx:eidx,sidx:eidx] = l * C  # variance over segment
    inseg = (startpt <= i) & (endpt >= i+1)  # True for forecasts that contain segment
    Z[inseg, sidx:eidx] = P[inseg,:]    # put coefficients on segment vars involved in forecasts

  # now have everything to calculate posterior distribution
  ZOmega = Z.dot(Omega)
  F = ZOmega.dot(Z.T) + H
  # B = Omega Z' inv(F), and F and Omega are symmetric
  # B = (ZOmega.T).dot(np.linalg.inv(F))
  B = np.linalg.solve(F, ZOmega).T  # computationally better this way

  # segment variables given forecasts have
  # mean = a_mean + B y   = nu + B (y - Z nu)   where y is the vector of forecasts
  # & cov = Sigma
  Sigma = Omega - B.dot(ZOmega)
  a_mean = nu - B.dot(Z.dot(nu))
  return Tpts, a_mean, B, Sigma

In [0]:
Tpts, a_mean, B, Sigma = calculate_posterior_of_segments(start, end, P, H, mu, C)

### 3. Now can calculate the mean and cov of any linear combinations of segment variables given the forecasts

In [38]:
m, n = P.shape
nTps = len(Tpts) 
nsegvars = B.shape[0]

print(nTps, "Tpts -", Tpts, "- so", nTps-1, "segments")
print(n, "securities,", nsegvars, "security segments")
print(m, "forecasts")
print("a_mean.shape:", a_mean.shape)
print("B.shape:", B.shape)
print("Sigma.shape", Sigma.shape)

# segment variables given forecasts have
# mean = a_mean + B y  and  cov = Sigma    where y is the vector of forecasts

8 Tpts - [ 0.  4.  5.  6.  7.  8.  9. 10.] - so 7 segments
20 securities, 140 security segments
5 forecasts
a_mean.shape: (140, 1)
B.shape: (140, 5)
Sigma.shape (140, 140)


#### A. Example: all the securities over the interval between the 2nd and 4th Tpt

In [0]:
A = np.zeros((n, nsegvars))
A[:,n:2*n] = np.identity(n)
A[:,2*n:3*n] = np.identity(n)

In [42]:
# given the forecasts, these segment variable combinations
# have mean m0 + M y and cov A Sigma A'
m0 = A.dot(a_mean)
M = A.dot(B)   # tells how much each forecast contributed
pmean = m0 + M.dot(y)
pcov = A.dot(Sigma).dot(A.T)
print(pmean.shape)
print(pcov.shape)

(20, 1)
(20, 20)


#### B. Example: the first security over each separate segment

In [0]:
nsegs = nTps-1
A = np.zeros((nsegs, nsegvars))
secnum = 0 # first security
for i in range(nsegs):
  A[i, i*n + secnum] = 1.   # each row in A is a different interval of the same security

In [60]:
# given the forecasts, these segment variable combinations
# have mean m0 + M y and cov A Sigma A'
m0 = A.dot(a_mean)
M = A.dot(B)   # tells how much each forecast contributed
pmean = m0 + M.dot(y)
pcov = A.dot(Sigma).dot(A.T)
print(pmean.shape)
print(pcov.shape)

(7, 1)
(7, 7)
